In [1]:
!pip3 install pandas
!pip3 install seaborn
!pip3 install --upgrade tensorflow-gpu
!pip3 install import-ipynb

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import math
import pickle
import time
import sys

import import_ipynb
import VariationalAutoencoderModel

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda, Input, Dense
from tensorflow.keras.losses import mse
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, mean_squared_error, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

from bayes_opt import BayesianOptimization

%matplotlib inline

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
importing Jupyter notebook from VariationalAutoencoderModel.ipynb
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


2022-08-16 19:01:59.139548: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-16 19:01:59.139578: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction = 0.333)
sess = tf.compat.v1.Session(config = tf.compat.v1.ConfigProto(gpu_options = gpu_options))

2022-08-16 19:02:00.198722: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-16 19:02:00.234784: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-16 19:02:00.234910: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-08-16 19:02:00.235023: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared obj

# Danmini Doorbell Data Pre-Processing

In [3]:
# Benign traffic

dd_benign = pd.read_csv('../nbaiot/Danmini_Doorbell/benign_traffic.csv', encoding = "utf-8", sep = ',' ) 
df_dd_benign = dd_benign.copy(deep=True)

columns = list(df_dd_benign.columns)
chosen_columns = []
for column in columns:
    if column.find('L5') != -1 or column.find('L3') != -1 or column.find('L1') != -1 or column.find('L=0.1') != -1 or column.find('L0.01') != -1:
        chosen_columns.append(column)
        
df_dd_benign = pd.DataFrame(df_dd_benign, columns = chosen_columns)

# Mirai

dd_mirai_ack = pd.read_csv('../nbaiot/Danmini_Doorbell/mirai/ack.csv', encoding = "utf-8", sep = ',' ) 
df_dd_mirai_ack = dd_mirai_ack.copy(deep=True)
df_dd_mirai_ack = pd.DataFrame(df_dd_mirai_ack, columns = chosen_columns)

dd_mirai_scan = pd.read_csv('../nbaiot/Danmini_Doorbell/mirai/scan.csv', encoding = "utf-8", sep = ',' ) 
df_dd_mirai_scan = dd_mirai_scan.copy(deep=True)
df_dd_mirai_scan = pd.DataFrame(df_dd_mirai_scan, columns = chosen_columns)

dd_mirai_syn = pd.read_csv('../nbaiot/Danmini_Doorbell/mirai/syn.csv', encoding = "utf-8", sep = ',' ) 
df_dd_mirai_syn = dd_mirai_syn.copy(deep=True)
df_dd_mirai_syn = pd.DataFrame(df_dd_mirai_syn, columns = chosen_columns)

dd_mirai_udp = pd.read_csv('../nbaiot/Danmini_Doorbell/mirai/udp.csv', encoding = "utf-8", sep = ',' ) 
df_dd_mirai_udp = dd_mirai_udp.copy(deep=True)
df_dd_mirai_udp = pd.DataFrame(df_dd_mirai_udp, columns = chosen_columns)

dd_mirai_udpplain = pd.read_csv('../nbaiot/Danmini_Doorbell/mirai/udpplain.csv', encoding = "utf-8", sep = ',' ) 
df_dd_mirai_udpplain = dd_mirai_udpplain.copy(deep=True)
df_dd_mirai_udpplain = pd.DataFrame(df_dd_mirai_udpplain, columns = chosen_columns)

# Bashlite

dd_bashlite_combo = pd.read_csv('../nbaiot/Danmini_Doorbell/gafgyt/combo.csv', encoding = "utf-8", sep = ',' ) 
df_dd_bashlite_combo = dd_bashlite_combo.copy(deep=True)
df_dd_bashlite_combo = pd.DataFrame(df_dd_bashlite_combo, columns = chosen_columns)

dd_bashlite_junk = pd.read_csv('../nbaiot/Danmini_Doorbell/gafgyt/junk.csv', encoding = "utf-8", sep = ',' ) 
df_dd_bashlite_junk = dd_bashlite_junk.copy(deep=True)
df_dd_bashlite_junk = pd.DataFrame(df_dd_bashlite_junk, columns = chosen_columns)

dd_bashlite_scan = pd.read_csv('../nbaiot/Danmini_Doorbell/gafgyt/scan.csv', encoding = "utf-8", sep = ',' ) 
df_dd_bashlite_scan = dd_bashlite_scan.copy(deep=True)
df_dd_bashlite_scan = pd.DataFrame(df_dd_bashlite_scan, columns = chosen_columns)

dd_bashlite_udp = pd.read_csv('../nbaiot/Danmini_Doorbell/gafgyt/udp.csv', encoding = "utf-8", sep = ',' ) 
df_dd_bashlite_udp = dd_bashlite_udp.copy(deep=True)
df_dd_bashlite_udp = pd.DataFrame(df_dd_bashlite_udp, columns = chosen_columns)

dd_bashlite_tcp = pd.read_csv('../nbaiot/Danmini_Doorbell/gafgyt/tcp.csv', encoding = "utf-8", sep = ',' ) 
df_dd_bashlite_tcp = dd_bashlite_tcp.copy(deep=True)
df_dd_bashlite_tcp = pd.DataFrame(df_dd_bashlite_tcp, columns = chosen_columns)

In [4]:
scaler = MinMaxScaler()
df_dd_benign_norm = scaler.fit_transform(df_dd_benign)

df_dd_miraiack_norm = scaler.fit_transform(df_dd_mirai_ack)
df_dd_miraiscan_norm = scaler.fit_transform(df_dd_mirai_scan)
df_dd_miraisyn_norm = scaler.fit_transform(df_dd_mirai_syn)
df_dd_miraiudp_norm = scaler.fit_transform(df_dd_mirai_udp)
df_dd_miraiudpplain_norm = scaler.fit_transform(df_dd_mirai_udpplain)

df_dd_bashlitecombo_norm = scaler.fit_transform(df_dd_bashlite_combo)
df_dd_bashlitejunk_norm = scaler.fit_transform(df_dd_bashlite_junk)
df_dd_bashlitescan_norm = scaler.fit_transform(df_dd_bashlite_scan)
df_dd_bashliteudp_norm = scaler.fit_transform(df_dd_bashlite_udp)
df_dd_bashlitetcp_norm = scaler.fit_transform(df_dd_bashlite_tcp)

# Variational Autoencoder - Attack Detection

## Model

In [5]:
# Train set

len_benign = len(df_dd_benign_norm)
len_benign_train = int(0.7 * len_benign)
X_train_VAE = df_dd_benign_norm[:len_benign_train]

# Test set - 30% benign and the rest is attack

X_test_benign = df_dd_benign_norm[len_benign_train:]

len_attack_test = len(X_test_benign)
X_test_mirai_ack = df_dd_miraiack_norm
X_test_mirai_scan = df_dd_miraiscan_norm
X_test_mirai_syn = df_dd_miraisyn_norm
X_test_mirai_udp = df_dd_miraiudp_norm
X_test_mirai_udpplain = df_dd_miraiudpplain_norm
X_test_bashlite_combo = df_dd_bashlitecombo_norm
X_test_bashlite_junk = df_dd_bashlitejunk_norm
X_test_bashlite_scan = df_dd_bashlitescan_norm
X_test_bashlite_tcp = df_dd_bashlitetcp_norm
X_test_bashlite_udp = df_dd_bashliteudp_norm

X_test_VAE = np.concatenate([X_test_benign, X_test_mirai_ack, X_test_mirai_scan, X_test_mirai_syn, X_test_mirai_udp,
                             X_test_mirai_udpplain, X_test_bashlite_combo, X_test_bashlite_junk, X_test_bashlite_scan,
                             X_test_bashlite_tcp, X_test_bashlite_udp])

Y_test_VAE = np.ones(len(X_test_VAE))
Y_test_VAE[:len(X_test_benign)] = 0

In [6]:
# KL Loss function

def vae_loss(x, x_decoded_mean):
    # Compute the average MSE error, then scale it up (sum on all axes)
    
    reconstruction_loss = K.sum(K.square(x - x_decoded_mean))
    
    # Compute the KL loss
    
    kl_loss = - 0.5 * K.sum(1 + z_var - K.square(z_mean) - K.square(K.exp(z_var)), axis=-1)
    
    # Return the average loss over all 
    
    total_loss = K.mean(reconstruction_loss + kl_loss) # Total_loss = reconstruction_loss + kl_loss 
    return total_loss

# (1) Reconstruction Loss - Forces the encoder to generate latent features that minimize the reconstruction error, or else is
# penalized
# (2) KL Loss - Forces the distribution generated by the encoder to be similar to the prior probability of the input vector, 
# pushing latent feature space to normality

In [7]:
# Parameters

original_dim = X_train_VAE.shape[1]
input_shape = (original_dim,)
intermediate_dim = int(original_dim / 2)
latent_dim = int(original_dim / 3)

epochs = 800
learning_rate = 0.0001
batch_size = 82
anomaly_threshold = 0.042
number_features = 115

dict_params = { 'learning_rate': learning_rate, 'batch_size': int(32 * batch_size), 'epochs': int(epochs)}
pbounds = { 'learning_rate': (0.000001, 0.1), 'batch_size': (1, 4.001), 'epochs': (100, 1000)}

In [8]:
# Encoder

inputs, encoder, z_var, z_mean = VariationalAutoencoderModel.vae_encoder(
    input_shape, 
    intermediate_dim, 
    latent_dim, 
    VariationalAutoencoderModel.sample)

encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 92)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 46)           4278        ['encoder_input[0][0]']          
                                                                                                  
 z_mean (Dense)                 (None, 30)           1410        ['dense[0][0]']                  
                                                                                                  
 z_var (Dense)                  (None, 30)           1410        ['dense[0][0]']                  
                                                                                            

In [9]:
# Decoder

decoder = VariationalAutoencoderModel.vae_decoder(
    intermediate_dim,
    latent_dim,
    original_dim)

outputs = decoder(encoder(inputs))

decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 z_sampling (InputLayer)     [(None, 30)]              0         
                                                                 
 dense_1 (Dense)             (None, 46)                1426      
                                                                 
 dense_2 (Dense)             (None, 92)                4324      
                                                                 
Total params: 5,750
Trainable params: 5,750
Non-trainable params: 0
_________________________________________________________________


## Train

In [10]:
def training(X_train = X_train_VAE,
             X_test = X_test_VAE,
             Y_test = Y_test_VAE,
             inputs = inputs,
             outputs = outputs,
             vae_loss = vae_loss,
             epochs = epochs,
             batch_size = batch_size,
             learning_rate = learning_rate):
    
    opt = optimizers.Adam(learning_rate = dict_params['learning_rate'], clipvalue = 0.5)
    
    vae_model = Model(inputs, outputs, name = 'vae_mlp')
    vae_model.compile(optimizer = opt, loss = vae_loss)
    
    history = vae_model.fit(X_train, X_train,
                            shuffle = True,
                            epochs = dict_params['epochs'],
                            batch_size = dict_params['batch_size'])
    
    X_pred = vae_model.predict(X_test)
    error_vector = VariationalAutoencoderModel.get_error_term(X_pred, X_test, _rmse=False)
    attacks = (error_vector > anomaly_threshold)
    
    accuracy = accuracy_score(Y_test, attacks)
    
    return accuracy

In [11]:
# Bayesian optimization to choose the best hyperparameters

optimizer = BayesianOptimization(
    f = training,
    pbounds = pbounds,
    verbose = 2, 
    random_state = 1,
)

train_start = time.time()

optimizer.maximize(init_points = 5, n_iter = 5)

train_end = time.time()
train_time = train_end - train_start
print("Training time:", train_time)

print(optimizer.max)

|   iter    |  target   | batch_... |  epochs   | learni... |
-------------------------------------------------------------
Train on 34683 samples
Epoch 1/800


2022-08-16 19:02:13.166046: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-08-16 19:02:13.178797: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


34683/34683 [==============================] - 0s 11us/sample - loss: 43749.3515
Epoch 2/800
34683/34683 [==============================] - 0s 5us/sample - loss: 42374.0139
Epoch 3/800
34683/34683 [==============================] - 0s 5us/sample - loss: 41047.2053
Epoch 4/800
34683/34683 [==============================] - 0s 4us/sample - loss: 39766.3979
Epoch 5/800
34683/34683 [==============================] - 0s 5us/sample - loss: 38533.9230
Epoch 6/800
34683/34683 [==============================] - 0s 4us/sample - loss: 37349.9512
Epoch 7/800
34683/34683 [==============================] - 0s 5us/sample - loss: 36170.5811
Epoch 8/800
34683/34683 [==============================] - 0s 5us/sample - loss: 35000.8770
Epoch 9/800
34683/34683 [==============================] - 0s 5us/sample - loss: 33829.3252
Epoch 10/800
34683/34683 [==============================] - 0s 4us/sample - loss: 32625.7322
Epoch 11/800
34683/34683 [==============================] - 0s 5us/sample - loss: 31406.47

34683/34683 [==============================] - 0s 5us/sample - loss: 372.4347
Epoch 91/800
34683/34683 [==============================] - 0s 5us/sample - loss: 366.5470
Epoch 92/800
34683/34683 [==============================] - 0s 5us/sample - loss: 363.0595
Epoch 93/800
34683/34683 [==============================] - 0s 4us/sample - loss: 358.5489
Epoch 94/800
34683/34683 [==============================] - 0s 5us/sample - loss: 355.0143
Epoch 95/800
34683/34683 [==============================] - 0s 4us/sample - loss: 349.3357
Epoch 96/800
34683/34683 [==============================] - 0s 5us/sample - loss: 345.4180
Epoch 97/800
34683/34683 [==============================] - 0s 4us/sample - loss: 340.8686
Epoch 98/800
34683/34683 [==============================] - 0s 4us/sample - loss: 336.3355
Epoch 99/800
34683/34683 [==============================] - 0s 5us/sample - loss: 333.2677
Epoch 100/800
34683/34683 [==============================] - 0s 5us/sample - loss: 327.8819
Epoch 101/8

34683/34683 [==============================] - 0s 4us/sample - loss: 146.8695
Epoch 180/800
34683/34683 [==============================] - 0s 5us/sample - loss: 146.2700
Epoch 181/800
34683/34683 [==============================] - 0s 5us/sample - loss: 145.3690
Epoch 182/800
34683/34683 [==============================] - 0s 4us/sample - loss: 144.6898
Epoch 183/800
34683/34683 [==============================] - 0s 4us/sample - loss: 144.7089
Epoch 184/800
34683/34683 [==============================] - 0s 5us/sample - loss: 143.7338
Epoch 185/800
34683/34683 [==============================] - 0s 4us/sample - loss: 143.0543
Epoch 186/800
34683/34683 [==============================] - 0s 4us/sample - loss: 142.5476
Epoch 187/800
34683/34683 [==============================] - 0s 5us/sample - loss: 142.0273
Epoch 188/800
34683/34683 [==============================] - 0s 4us/sample - loss: 141.7962
Epoch 189/800
34683/34683 [==============================] - 0s 5us/sample - loss: 140.6118
Ep

...

34683/34683 [==============================] - 0s 4us/sample - loss: 15.3676
Epoch 541/800
34683/34683 [==============================] - 0s 5us/sample - loss: 15.4033
Epoch 542/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.3732
Epoch 543/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.3855
Epoch 544/800
34683/34683 [==============================] - 0s 5us/sample - loss: 15.3939
Epoch 545/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.3802
Epoch 546/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.3801
Epoch 547/800
34683/34683 [==============================] - 0s 5us/sample - loss: 15.3607
Epoch 548/800
34683/34683 [==============================] - 0s 5us/sample - loss: 15.3396
Epoch 549/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.3739
Epoch 550/800
34683/34683 [==============================] - 0s 5us/sample - loss: 15.3882
Epoch 551/800

34683/34683 [==============================] - 0s 5us/sample - loss: 15.3411
Epoch 631/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.3355
Epoch 632/800
34683/34683 [==============================] - 0s 5us/sample - loss: 15.3192
Epoch 633/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.3465
Epoch 634/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.3355
Epoch 635/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.3449
Epoch 636/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.3820
Epoch 637/800
34683/34683 [==============================] - 0s 5us/sample - loss: 15.3524
Epoch 638/800
34683/34683 [==============================] - 0s 5us/sample - loss: 15.3259
Epoch 639/800
34683/34683 [==============================] - 0s 5us/sample - loss: 15.3276
Epoch 640/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.3702
Epoch 641/800

34683/34683 [==============================] - 0s 4us/sample - loss: 15.3592
Epoch 721/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.2817
Epoch 722/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.2758
Epoch 723/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.2721
Epoch 724/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.2735
Epoch 725/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.3366
Epoch 726/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.2969
Epoch 727/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.3203
Epoch 728/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.3058
Epoch 729/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.2695
Epoch 730/800
34683/34683 [==============================] - 0s 4us/sample - loss: 15.3371
Epoch 731/800

/home/andressa.amaral/.local/lib/python3.7/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


|  10       |  1.0      |  3.595    |  878.6    |  0.04997  |
Training time: 1605.9569926261902
{'target': 0.9999878001047158, 'params': {'batch_size': 1.5589668943443904, 'epochs': 411.00465433874297, 'learning_rate': 0.03967735065559277}}


In [12]:
# Training step with the best hyperparameters

opt = optimizers.Adam(learning_rate = optimizer.max['params']['learning_rate'], clipvalue = 0.5)

vae_model = Model(inputs, outputs, name = 'vae_mlp')
vae_model.compile(optimizer = opt, loss = vae_loss)
vae_model.summary()

train_start = time.time()

history = vae_model.fit(X_train_VAE, X_train_VAE,
                            shuffle = True,
                            epochs = int(optimizer.max['params']['epochs']),
                            batch_size = int(32 * optimizer.max['params']['batch_size']))

train_end = time.time()
train_time = train_end - train_start
print("Training time:", train_time)

Model: "vae_mlp"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 92)]              0         
                                                                 
 encoder (Functional)        (None, 30)                7098      
                                                                 
 decoder (Functional)        (None, 92)                5750      
                                                                 
Total params: 12,848
Trainable params: 12,848
Non-trainable params: 0
_________________________________________________________________
Train on 34683 samples
Epoch 1/411
34683/34683 [==============================] - 2s 58us/sample - loss: 30.3595
Epoch 2/411
34683/34683 [==============================] - 2s 51us/sample - loss: 5.4690
Epoch 3/411
34683/34683 [==============================] - 2s 50us/sample - loss: 14.5104
Epoch 4/411
34683/34683 [==

34683/34683 [==============================] - 2s 50us/sample - loss: 10049105502.8873
Epoch 83/411
34683/34683 [==============================] - 2s 50us/sample - loss: 23018.1733
Epoch 84/411
34683/34683 [==============================] - 2s 50us/sample - loss: 7.7924
Epoch 85/411
34683/34683 [==============================] - 2s 51us/sample - loss: 5.4526
Epoch 86/411
34683/34683 [==============================] - 2s 52us/sample - loss: 5.4863
Epoch 87/411
34683/34683 [==============================] - 2s 50us/sample - loss: 5.3822
Epoch 88/411
34683/34683 [==============================] - 2s 47us/sample - loss: 5.4229
Epoch 89/411
34683/34683 [==============================] - 2s 49us/sample - loss: 5.3709
Epoch 90/411
34683/34683 [==============================] - 2s 50us/sample - loss: 5.3627
Epoch 91/411
34683/34683 [==============================] - 2s 49us/sample - loss: 5.3486
Epoch 92/411
34683/34683 [==============================] - 2s 51us/sample - loss: 5.4252
Epoch 93/

34683/34683 [==============================] - 2s 50us/sample - loss: 5.5110
Epoch 171/411
34683/34683 [==============================] - 2s 51us/sample - loss: 13.8213
Epoch 172/411
34683/34683 [==============================] - 2s 49us/sample - loss: 2350.2868
Epoch 173/411
34683/34683 [==============================] - 2s 51us/sample - loss: 5.8345
Epoch 174/411
34683/34683 [==============================] - 2s 51us/sample - loss: 213449685749.4428
Epoch 175/411
34683/34683 [==============================] - 2s 52us/sample - loss: 9277.8710
Epoch 176/411
34683/34683 [==============================] - 2s 50us/sample - loss: 5.5540
Epoch 177/411
34683/34683 [==============================] - 2s 51us/sample - loss: 13611093.6160
Epoch 178/411
34683/34683 [==============================] - 2s 50us/sample - loss: 96002616.6450
Epoch 179/411
34683/34683 [==============================] - 2s 52us/sample - loss: 127883.3451
Epoch 180/411
34683/34683 [==============================] - 2s 49u

34683/34683 [==============================] - 2s 50us/sample - loss: 5.4775
Epoch 257/411
34683/34683 [==============================] - 2s 50us/sample - loss: 5.4750
Epoch 258/411
34683/34683 [==============================] - 2s 50us/sample - loss: 16512140006789.9258
Epoch 259/411
34683/34683 [==============================] - 2s 50us/sample - loss: 12933334759.8292
Epoch 260/411
34683/34683 [==============================] - 2s 50us/sample - loss: 177280.7045
Epoch 261/411
34683/34683 [==============================] - 2s 50us/sample - loss: 6.8789
Epoch 262/411
34683/34683 [==============================] - 2s 50us/sample - loss: 6.4674
Epoch 263/411
34683/34683 [==============================] - 2s 50us/sample - loss: 5.4902
Epoch 264/411
34683/34683 [==============================] - 2s 50us/sample - loss: 5.4728
Epoch 265/411
34683/34683 [==============================] - 2s 51us/sample - loss: 6.1799
Epoch 266/411
34683/34683 [==============================] - 2s 49us/sample 

34683/34683 [==============================] - 2s 50us/sample - loss: 5.5277
Epoch 344/411
34683/34683 [==============================] - 2s 50us/sample - loss: 5.5933
Epoch 345/411
34683/34683 [==============================] - 2s 50us/sample - loss: 5.4909
Epoch 346/411
34683/34683 [==============================] - 2s 50us/sample - loss: 24157.2422
Epoch 347/411
34683/34683 [==============================] - 2s 50us/sample - loss: 76273361837547947471821365837824.0000
Epoch 348/411
34683/34683 [==============================] - 2s 50us/sample - loss: 184313439997686659252486144.0000
Epoch 349/411
34683/34683 [==============================] - 2s 49us/sample - loss: 249240491962405280.0000
Epoch 350/411
34683/34683 [==============================] - 2s 49us/sample - loss: 2256661497065472256.0000
Epoch 351/411
34683/34683 [==============================] - 2s 50us/sample - loss: 2495225123667663.0000
Epoch 352/411
34683/34683 [==============================] - 2s 50us/sample - loss: 

In [13]:
plt.plot(results.history['loss'])
plt.title('Loss Pattern')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Test'], loc='upper right');
plt.show()

NameError: name 'results' is not defined

## Test

In [ ]:
X_train_pred = vae_model.predict(X_train_VAE)
error_vector = VariationalAutoencoderModel.get_error_term(X_train_pred, X_train_VAE, _rmse=False)
error_thresh = anomaly_threshold

# Measure the error between the train set and its output and generates an error vector representing the error of each term
# Find an extreme value on the vector to use as the error threshold

In [ ]:
test_start = time.time()

X_pred_VAE = vae_model.predict(X_test_VAE)

test_end = time.time()
test_time = test_end - test_start
print("Testing time:", test_time)

# If the error of the vector is higher than the defined threshold it detects an attack

error_vector = VariationalAutoencoderModel.get_error_term(X_pred_VAE, X_test_VAE, _rmse=False)
attacks = (error_vector > error_thresh)
print(attacks)

np.count_nonzero(attacks) / len(attacks) # Percentage of attacks

In [ ]:
print(len(X_test_VAE))

## Metrics

In [ ]:
sys.stdout = open("../Results/damini_doorbell.txt", "a")

print(" ==== Test " + str(number_features) + " features - Normal learning rate ====")
print("Training time:" + str(train_time))
print("Testing time:" + str(test_time))
print(classification_report(Y_test_VAE, attacks, digits = 5))

In [ ]:
'''# Examinig the latent space generated by the encoder

X_encoded = encoder.predict(X_test_VAE)

pca = PCA(n_components=2)
X_analysis = pca.fit_transform(X_encoded)'''

In [ ]:
'''plt.figure(figsize = (12, 10))
sns.scatterplot(x = X_analysis[:, 0], y = X_analysis[:, 1], s = 20, hue = error_vector)
plt.grid()
plt.show()

# Darker clusters indicate that the error is higher'''

In [ ]:
'''plt.figure(figsize=(12, 10))
sns.scatterplot(x = X_analysis[:, 0], y = X_analysis[:, 1], s = 20, hue = attacks)
plt.grid()
plt.show()

# Orange ones are anomalies'''

In [ ]:
'''plt.figure(figsize = (12, 10))
sns.scatterplot(x = X_analysis[:, 0], y = X_analysis[:, 1], s = 20, hue = Y_test_VAE)
plt.grid()
plt.show()

# Ground truth'''